In [ ]:
from llama_index.core import StorageContext,load_index_from_storage
from llama_index.core import get_response_synthesizer
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.llms.openai_like import OpenAILike
from llama_index.core import Settings
from transformers import AutoTokenizer

custom_api = 'float16-56hBpTpGA5VYSs13Awe9U1FjGaTo5pNODx67cnI066EBP5rewr'
custom_url = 'https://api.float16.cloud/v1/llamaindex/'
tokenizer = AutoTokenizer.from_pretrained("SeaLLMs/SeaLLM-7B-v2").encode
llm = OpenAILike(api_base=custom_url,
                 api_key=custom_api,
                 max_tokens=512,
                 model='SeaLLM-7b-v2',
                 api_version='v1',
                 )

embed_model = HuggingFaceEmbedding(model_name="intfloat/multilingual-e5-large",device='cpu')
index = load_index_from_storage(StorageContext.from_defaults(persist_dir='./localstore/'),embed_model=embed_model)

In [ ]:
retrieval_engine = index.as_retriever(similarity_top_k = 1)
responseEmbed = retrieval_engine.retrieve("อุเทสิกเจดีย์คืออะไร")
OutEmbed = responseEmbed[0].text
print(OutEmbed)

In [ ]:
Settings.embed_model = embed_model
Settings.llm = llm
Settings.context_window = 4000 
Settings.tokenizer = tokenizer
Settings.num_output = 512
response_synthesizer = get_response_synthesizer(
      response_mode="simple_summarize"
)

query_engine = index.as_query_engine(
    llm=llm,
    similarity_top_k = 1,
    response_synthesizer = response_synthesizer)
response = query_engine.query("<summarization_query_thai>")

In [ ]:
print(response)